In [ ]:
import string
import wikipedia
from rank_bm25 import BM25Okapi
import pymorphy2
from natasha import MorphVocab
from razdel import tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import razdel
import nltk
import string
from string import punctuation
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util
import unicodedata
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from IPython.display import clear_output
import tensorflow as tf
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,

    Doc

)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def delete_punctuation(sen):
  remove_punc = string.punctuation + '«»' 
  return ''.join([i for i in sen if (i not in remove_punc) or i == '-']) #return string

In [ ]:
def word_morphy(sen, morph = pymorphy2.MorphAnalyzer()):
  return ' '.join([morph.parse(_.text)[0].normal_form for _ in tokenize(sen)])

In [ ]:
def ner_for_search(claim, emb = NewsEmbedding(), segmenter = Segmenter()):
  corpus = []
  ner_tagger = NewsNERTagger(emb)
  doc = Doc(claim)
  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)
  for spans in doc.spans:
    corpus = corpus + wikipedia.search(spans.text)
  return corpus

In [ ]:
# input: usial sen
# output: list of top k-topics
# n - results of wiki.search
# k - results of top k-topics
# function is okey
def top_k_articles(query, n = None, k = 2, lang = 'ru', need_score = False):
  corpus, corpus_ner = [],[]
  wikipedia.set_lang(lang)
  if len(query) < 300:
    corpus = wikipedia.search(query, results = 3)
  corpus_ner = ner_for_search(query)
  if type(corpus) is list:
    if type(corpus_ner) is list:
      new_corpus = corpus + corpus_ner
    else: new_corpus = corpus
  elif type(corpus_ner) is list: new_corpus = corpus_ner
  else: return 'Недостаточно информации'
  if len(new_corpus) != 0:
    new_corpus_pre = list(map(delete_punctuation, new_corpus)) #мб токены сделать сразу
    new_corpus_pre = list(map(word_morphy, new_corpus_pre))
    tokenized_corpus = [doc.split(" ") for doc in new_corpus_pre]
    tokenized_query = query.split(" ")
    tokenized_query = word_morphy(delete_punctuation(query)).split(" ")
   
    bm25 = BM25Okapi(tokenized_corpus)
    score = bm25.get_scores(tokenized_query)
    lst_article_score = []
    for i in range(len(score)):
      if score[i] > 0:
          lst_article_score.append([new_corpus[i], score[i]])

    lst_article_score.sort(key= lambda x: x[1], reverse=True)
    #print('print', lst_article_score)
    article = [element[0] for element in lst_article_score]
    return lst_article_score if need_score == True else article[:k] 
  else: return 'Недостаточно информации'

In [ ]:
def get_token_sen(page):
    start = re.search(r'\n{3}===? Примечания ===?', page)
    start_next = re.search(r'\n{3}===? Издания ===?', page)
    if start is re.Match:
      step = start.span()[0]
      page = page[:step]
    if start_next is re.Match:
      step_next = start_next.span()[0]
      if start > start_next:
        page = page[:start_next]

    clean = re.sub('\n{3}===? [а-яА-ЯЁё0-9.,-a-zA-Z ]+ ===?',' ',page) #[] добавить символы типо -/?
    #token
    return [_.text for _ in razdel.sentenize(clean)]

In [ ]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [ ]:
def token_word_morphy(sen, morph = pymorphy2.MorphAnalyzer()):
  return [morph.parse(_.text)[0].normal_form for _ in razdel.tokenize(sen)] 

In [ ]:
def noise():
    return (stopwords.words('russian') + list(punctuation + 'IVIX«»\n№́'))

In [ ]:
#'DeepPavlov/rubert-base-cased'
def rubert_cosine_sim_score(claim, page, min_value = 0.35, top = 5,\
                            model = SentenceTransformer('distiluse-base-multilingual-cased-v1')):
  lst_sen_score = []

  text = get_token_sen(page)
  text = list(map(remove_accents, text))
  text.insert(0, claim)

  embeddings = model.encode(text, convert_to_tensor=True)
  for i in range(1, len(embeddings)):
    score = util.pytorch_cos_sim(embeddings[0], embeddings[i])
    if score > min_value:
      lst_sen_score.append([i, text[i], score])
  lst_sen_score.sort(key= lambda x: x[2], reverse=True)
  if top is not None:
    return lst_sen_score[:top]
  else: return lst_sen_score

In [ ]:
def top_l_sentences(claim, corpus_articles, top_sen = 20, only_sen = True):
  wikipedia.set_lang("ru")
  lst_sen = []
  if len(corpus_articles) == 0: return 'Недостаточно информации'
  for article in corpus_articles:
    try:
      complete_content = wikipedia.page(article)
      page = complete_content.content
      lst_sen.append(rubert_cosine_sim_score(claim, page))
    except Exception: continue
  if type(lst_sen) is not list: return 'Недостаточно информации'
  if only_sen == False: return lst_sen
  else: 
    lst = []
    for i_article in lst_sen:
      for sentences in i_article:
        lst.append(sentences[1])
    if len(lst) > top_sen:
      return lst[:top_sen]
    else: return lst

In [ ]:
def nli_componet(claim, corpus_sen):
  array_score, array_label = [], []
  for i_sentence in corpus_sen:
    prediction_score, prediction_label = check_sentences(claim, i_sentence)
    array_score = np.append(array_score, prediction_score).reshape(-1,2)
    array_label = np.append(array_label, prediction_label)
  return array_score, array_label

In [ ]:
def logical_aggregation(prediction_label, prediction_score, corpus_sen):
  max_pred = prediction_score.max()
  s = sum(prediction_label)
  r = abs(len(prediction_label) - s)
  if r == 0:
    return 'Правда', corpus_sen[int(np.where(prediction_score == max_pred)[0])]
  elif s == 0:
    #print('hi')
    return 'Ложь', corpus_sen[int(np.where(prediction_score == max_pred)[0])]
  elif r == 0 and s == 0:
    return 'Недостаточно информации'
  else: 
    index = np.argmax(prediction_score)
    #print(max_pred)
    if index % 2 == 0:
      return 'Ложь', corpus_sen[int(np.where(prediction_score == max_pred)[0])], 'Вероятность {}%'.format(round(max_pred*100,2))
    else: return 'Правда', corpus_sen[int(np.where(prediction_score == max_pred)[0])], 'Вероятность {}%'.format(round(max_pred*100,2))

In [ ]:
def check_sentences(sen1, sen2, check=False, model = torch.load('/content/drive/MyDrive/model_RuBert3.pt', map_location=torch.device('cpu'))):
  in_id, token_id, mask = encoded_inputs_(sen1,sen2)
  inputs = torch.tensor([in_id])
  token = torch.tensor([token_id])
  masks = torch.tensor([mask])  
  outputs = model(input_ids = inputs, token_type_ids = token, attention_mask = masks)
  prob = tf.nn.softmax(outputs[0].detach().numpy()).numpy()[0]

  prob_index = np.argmax(prob)

  if check == True: return 'Не верно' if prob_index == 0 else 'Верно'
  else: return prob, prob_index

In [ ]:
from transformers import AutoTokenizer
#input: str, str
#output: input_ids, token_type_ids, attention_mask
#tokenizer для мультиязычного: 'bert-base-multilingual-cased'
#только для русского языка: 'DeepPavlov/rubert-base-cased'
def encoded_inputs_(claim, evidence, tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased'), MAXLEN = 512):
  encoded_inputs = tokenizer(claim, evidence, max_length=MAXLEN)
  return encoded_inputs['input_ids'], encoded_inputs['token_type_ids'], encoded_inputs ['attention_mask']

In [ ]:
def fact_checking(claim):
  score, label = 0, 0
  corpus_articles = top_k_articles(claim)
  if corpus_articles == 'Недостаточно информации': return 'Недостаточно информации'
  corpus_sen = top_l_sentences(claim, corpus_articles)
  if corpus_sen == 'Недостаточно информации' or len(corpus_sen) == 0: return 'Недостаточно информации'
  #print(corpus_sen)
  try:
    score, label = nli_componet(claim, corpus_sen)
    return logical_aggregation(label,score,corpus_sen)
  except ValueError:
    return 'Недостаточно информации'

In [ ]:
fact_checking('Атлантический палтус — самый крупный представитель камбалообразных рыб. ')

('Правда',
 'Атлантическии белокорыи палтус или атлантическии палтус  (лат. Hippoglossus hippoglossus) — рыба семеиства камбаловых, один из самых крупных представителеи семеиства.',
 'Вероятность 89.37%')

In [ ]:
fact_checking('В мире насчитывается около 200 пород кошек')

('Правда',
 'В настоящее время в мире насчитывается около 600 млн домашних кошек, выведено около 200 пород, от длинношерстных (персидская кошка) до лишенных шерсти (сфинксы), признанных и зарегистрированных различными фелинологическими организациями.')